In [ ]:
import pandas as pd
import re
import pickle
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.llms.openai import OpenAIChat
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

In [ ]:
api_key = "YOUR OPENAI API KEY"

In [ ]:
df = pd.read_excel('Transcription Excel data.xlsx')

In [ ]:
document_loader = PyMuPDFLoader('DSM-5 Pdf.pdf')
document = document_loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len
)

documnet_chunks = text_splitter.transform_documents(document)

In [ ]:
store = LocalFileStore('./document_cache/')

# create an embedder
#text-embedding-ada
core_embeddings_model = OpenAIEmbeddings(openai_api_key=api_key) 

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace = core_embeddings_model.model
)
# store embeddings in vector store
vectorstore = FAISS.from_documents(documnet_chunks, embedder)

In [ ]:
# instantiate a retriever
retriever = vectorstore.as_retriever()

In [ ]:
# OpenAI GPT-4 8k 
llm = OpenAIChat(openai_api_key=api_key,model_name="gpt-4")
handler =  StdOutCallbackHandler()

template = """아래의 내용에 기반하여 질문에 답하여라:
{context}

질문: 정신건강의학적으로 유의미한 증상에는 [각 환자의 증상 목록]가 있다. 다음 I와 환자(P7,P9...)의 상담 내용을 보고 정신건강의학적으로 유의미한 증상이 있다면 해당 증상과 이를 시사하는 구획을 추출하라. 만약 유의미한 증상이 없다면 '해당없음'이라고 대답하라. 
- I와 환자(P7,P9...)의 상담 내용 : {question}

답변:
- 증상 :
- 구획 :
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

chain_type_kwargs = {"prompt":prompt}

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
def RAG(df):
    
    df['Estimated_Symptom'] = ''
    df['Estimated_Section'] = ''
    
    for idx,row in df.iterrows():
        
        question = row['Statement']
        
        response = qa_chain({'query':question})
        
        with open(f'rag_dsm5_results_p7/result_{idx}.pkl','wb') as f:
            pickle.dump(response,f)
        
        symptom = re.search(r'- 증상 : (.*)', response['result'])
        section = re.search(r'- 구획 : (.*)', response['result'])
        
        documents = [i for i in response['source_documents']]
        
        if symptom == None:
        
            df.at[idx,'Estimated_Symptom'] = '해당없음'
            df.at[idx,'Estimated_Section'] = '해당없음'
            
        if symptom != None:
            
            try:
                df.at[idx,'Estimated_Symptom'] = symptom.group(1).lstrip(': ').strip()
                
            except:
                df.at[idx,'Estimated_Symptom'] = '해당없음'
            
            try:
                df.at[idx,'Estimated_Section'] = section.group(1).lstrip(': ').strip()
            
            except:
                df.at[idx,'Estimated_section'] = '해당없음'
            
    return df
        

In [ ]:
result = RAG(df)
result.to_excel('RAG Excel data.xlsx',index=False)